# Kerasを用いたCIFAR-10 チュートリアル

## 1. はじめに

このチュートリアルでは、[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)と呼ばれる画像分類問題を扱います。CIFAR-10は5万枚の32x32ピクセルのカラーの画像データと、それを分類する10個のラベル「飛行機、自動車、鳥、猫、鹿、犬、蛙、馬、船、トラック」で成り立っています。より細分化されたラベルに分類するCIFAR-100というものもあり、こちらはラベルが100個ついています。どちらも基本的には同じものなので、今回はCIFAR-10を使ってみます。

また、実装には[Keras](https://keras.io/ja/)を使います。Kerasは教師なし学習には使えませんが、CIFAR-10やMNISTを代表する典型的な機械学習の問題を扱う上においては、Kerasで十分だと思います。さらにKerasはシンプルなネットワーク（といっても一方向のネットワークだけではなく、分岐するようなもの、RNNなど）に関しては非常に簡単に実装できるので、初心者の方も（生の[Tensorflow](https://www.tensorflow.org)よりは）何やっているか理解しやすいと思います。このチュートリアルでは、Kerasの実装法も（簡単にですが）説明しているので、Kerasを知らない方も読み進めることができるのではないかと思います。詰まった際は[公式ドキュメント](https://keras.io/ja/)を参照ください。

このチュートリアルは私が[大学の講義](https://github.com/hiroyuki827/deep_learning_in_physics_research_SS17)で学んだ内容をベースにしているので、私自身が理解するのに時間がかかったところなどは余計に説明を加えました。全体的に冗長な感じは否めませんが、あえて残すことで似たような悩みを持つ方に役立つのではないかと考えています。

<u>予備知識&想定読者</u>
- 深くは理解できていないが、[MNIST for beginners](https://www.tensorflow.org/get_started/mnist/beginners)はやってみた。が、その後が続かない。
- 機械学習の背景については一通り勉強してみた（「[ゼロから作るDeep Learning](https://www.oreilly.co.jp/books/9784873117584/)」)
- Kerasを使ってCNNをやってみたい。

<u>動作環境</u>

出来る限りパワーのあるパソコンがあればいいですが、もしなければクラウドのサービス（たとえば[Google Colaboratry](https://qiita.com/tomo_makes/items/f70fe48c428d3a61e131)など）をどうぞライブラリのバージョンは以下の通り:

In [1]:
import numpy
numpy.__version__

'1.14.0'

In [2]:
import matplotlib
matplotlib.__version__

'2.1.2'

In [4]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.3'

### このノートを始める前に
<u>注意 その0</u> `dlt`パッケージについては、`pip install dlt`でダウンロードしてください。dltパッケージの使い方については、別途[`dltパッケージの使い方/`](https://github.com/hiroyuki827/deep_learning_tools/blob/master/dltパッケージの使い方/dliprパッケージの使い方.ipynb)を参照ください。Jupyter上では以下を実行してください。

In [ ]:
!pip install dlt

<u>注意 その1</u> 
いくつか勉強不足の点があるので、間違いを見つけられた場合はご指摘いただけると嬉しいです。これはissueページにて受け付けております。なお、各ライブラリのバージョンについては以下のようになっています。

<u>注意 その2</u> また、Jupyter notebookの行番号が入っていないところもありますが、基本的には上から読んでいっていただければと思います。本論については実際に実行し、結果をHTMLで記述しています。

## 2. データの読み込み

ここではdltを用いたデータの読み込みについて議論します。CIFAR-10のデータセットがどういう風に成り立っているのかを詳しく調べた後、one-hotベクトルについて説明しました。このあたりはご存じの方は軽く流していただいて構いません。

まず、必要なパッケージ等をインポートします。画像を扱うためには、その向きを考慮するためにCNN (Convolutional Neuron Network) を使う必要があるので、それを考慮して以下のようにインポートします。ここで`os`モジュールは、ファイルの保存ディレクトリの作成に使います。

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import dlt
import os

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

次にCIFAR-10のデータフォルダから必要なデータを読み込みます。後の学習結果の可視化を使うため、dltパッケージを用いてダウンロードします。

**コメント** データセットは`~/.keras/dataset`以下にダウンロードされます。ダウンロードにはTensorflowのコードを用いています。詳しくはdltパッケージ内の`cifar.py`参照。

In [6]:
data = dlt.cifar.load_cifar10()

In [7]:
#学習用の画像とラベル
print(data.train_images.shape)
print(data.train_labels.shape)

#テスト用の画像とラベル
print(data.test_images.shape)
print(data.test_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


今回は、訓練用として5万枚の画像ファイルと、テスト用の1万枚の画像ファイルを使います。それぞれの画像は32x32の1024画素で構成されています。要するに

`(画像, 行(or列)方向のピクセル, 列(or行)方向のピクセル, RGB+黒)`

のようになっています。

### 2.1 読み込んだデータの内容

ここで少し本筋からそれますが、読み込んだデータがどのようなものかを確認しておきたいと思います。
#### data.train_images

一つ目の添字`50000`には、各画像のRGB値が入っています。一つ目の画像のデータは以下のように見れます。

In [8]:
print(data.train_images[0]) #一つ目の添字`50000`のうち、一つ目の画像のデータ

[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]


二つ目と三つ目(`32, 32`)は、各画像のピクセルを指定するものです。各画像は32x32で構成されているので、この二つの添字を指定すると、どのピクセルを見ているかがわかります。たとえば一つ目の画像の、一つ目のピクセルを見たかったらまず、

In [9]:
print(data.train_images[0][0]) # [画像の番号][ピクセルの行or列方向指定]
print(data.train_images[0][0].shape)

[[ 59  62  63]
 [ 43  46  45]
 [ 50  48  43]
 [ 68  54  42]
 [ 98  73  52]
 [119  91  63]
 [139 107  75]
 [145 110  80]
 [149 117  89]
 [149 120  93]
 [131 103  77]
 [125  99  76]
 [142 115  91]
 [144 112  86]
 [137 105  79]
 [129  97  71]
 [137 106  79]
 [134 106  76]
 [124  97  64]
 [139 113  78]
 [139 112  75]
 [133 105  69]
 [136 105  74]
 [139 108  77]
 [152 120  89]
 [163 131 100]
 [168 136 108]
 [159 129 102]
 [158 130 104]
 [158 132 108]
 [152 125 102]
 [148 124 103]]
(32, 3)


として行or列方向のピクセルの情報を出します。これは32個あります。そして

In [10]:
print(data.train_images[0][0][0]) # [画像の番号][ピクセルの行or列方向指定][ピクセルの列or行方向指定]

[59 62 63]


とします。ここで`3`はRGBの自由度3を表します。つまり、このピクセルはRGBが`[59 62 63]`で構成されているということがわかります。

以上の考察ではそれぞれの画像を特定しましたが、画像を特定せずに、ピクセルすべてを見たいときは添字2つめと3つめを指定してやれば良いです。この場合、各ピクセルの3色の自由度も含まれることになります。

#### data.train_labels

次にラベルについて見ていきます。はじめの１枚の画像を調べると、

In [11]:
print(data.train_labels[0])

[6]


と得られます。これは`cifar.py`において

```
cifar10_labels = np.array([
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'])
```
とラベルが定義されているので、この画像は`frog`であることがわかります。

`data.test_images`, `data.test_labels`についても同様にしてわかります。

話をもとに戻しましょう。ここで、訓練データの画像を`examples.png`ファイルとして見てみます。(ここでは保存されたファイルを表示させています。)

In [12]:
dlt.utils.plot_examples(data, fname='examples.png')

<img src='examples.png'/>

入力値の訓練用データの変数、テスト用データの変数をそれぞれ`X_train`, `X_test`とおくことにしましょう。（単回帰解析で変数を$x$と置くことに対応します。)

In [13]:
X_train = data.train_images.reshape([-1, 32, 32, 3])
X_test = data.test_images.reshape([-1, 32, 32, 3])

print('%i training samples' % X_train.shape[0])
print('%i test samples' % X_test.shape[0])

print(X_train.shape)
print(X_test.shape)

50000 training samples
10000 test samples
(50000, 32, 32, 3)
(10000, 32, 32, 3)


※ もともとのデータ`data.train_images`, `data.test_labels`と同じサイズなのでreshapeする必要はありませんが、とりあえずしておきます。

##### データ量のラベルに対する分布

今回の場合はデータセットが用意されているので気にする必要はありませんが、自分でデータセットを作る際は、各ラベルに対してデータの偏りがないようにする必要があります。これについても次のようにチェックすることができます。

In [14]:
# Plot data distribution
dlt.utils.plot_distribution_data(Y=data.train_labels,
                                 dataset_name='y_train',
                                 classes=data.classes,
                                 fname='dist_train.png')

Mean Value: 5000
Median Value: 5000.0
Variance: 0
Standard Deviation: 0.0


<img src='dist_train.png'/>

このように、各ラベルに対して同じだけのデータ（厳密に言えば正解ラベル）が存在していることがわかります。実際、データセットを自分で作るときにこのようにしないと、最終的な結果にラベルのバイアスがかかってしまいます。

・・・このままでは各データを扱いにくいので、RGBの値(白なら255、黒なら0)を利用して0から1までの間に**正規化**することにします。この操作により`X_train, X_test`の形状は変わりません。

In [15]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

##### one-hotベクトルについて

ところで、分類問題では最後のActivationにsoftmaxを使います。というのは、出力値のリストの添字を見ることで、そのデータがどのカテゴリーに分類されうるか(その割合or確率)がわかるからです。たとえば適当な5つのカテゴリー問題を考えたとして

```
[0.03, 0.05, 0.85, 0.03, 0.04]
```
なるリストが得られたとすると、`np.argmax([0.03, 0.05, 0.85, 0.03, 0.04])=[2]`であるから結果として「カテゴリー`2`の確率が0.85x100=85%程度であるらしい」ことがわかります。

このように添字を一番大きいところを取ればそのカテゴリーがわかります。逆に言えば、正解ラベルだけ任意の数字を振ってほかと区別できるようにしておけば、そのラベルを見つければ良いのでコンピュータもそれがカテゴリーであることがわかりますよね。と考えると、正解ラベルに1, それ以外に0を振っておけば、1がある場所その添字を見れば正解かどうか分かることになります。これをone-hotベクトルと言います。具体例で考えてみます。

上で表したリストを例に取って、正解ラベルを2であるとすると、このときのone-hotベクトルは
```
[0, 0, 1, 0, 0]
```
となります。`1`のある場所`2`がその画像の属するカテゴリーであることを示しています。今回は10種類のカテゴリーの分類を扱うので、10個の要素を持つone-hotベクトルに分類しておきます。(いくつの分類がほしいかはここで決める。)

`one_hot`ベクトルへの変換メソッドは`dlt`内にも用意されています。これはNumpyでも実装できるんですね。ただまぁKerasでも`to_categorical`が用意されているので、こちらを使うことにします。ちなみにKerasの`to_categorical`はNumpyオブジェクトとして出力されるので、同じものです。

まず読み込んだ生のデータを見てみますと、

In [16]:
print(data.train_labels[:10]) #はじめの10個の画像ラベル

[[6]
 [9]
 [9]
 [4]
 [1]
 [1]
 [2]
 [7]
 [8]
 [3]]


となっています。一枚目の画像`6`については先程見たとおり`frog`でしたね。それではこれをone-hotベクトルに直すとはどういうことかを考えてみます。ラベルが`6`というのは、「10個の要素を持つタプル（カテゴリーが10個あるので)のうち、添字6番目の要素が`1`, それ以外が`0`」ということを意味しています。つまり、`[6]`というのは

```
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
```

というone-hotベクトルに対応します。このようにすれば、テスト用のデータをネットワークに通して予測させた後(`predict`)の結果は、その確率が一番大きいところ、すなわち`1`の場所がどこにあるかを調べれば良いことになりますね。このようにone-hotベクトルになおしておけば、softmaxからも解釈が可能になりカテゴリーの概念がよりコンピュータに理解させやすくなります。

※出力値も入力値に対応させて`Y_train`, `Y_test`と置くことにします。

ということで、訓練用`Y_train`とテスト用`Y_test`の出力値をone-hotベクトルに直しておきます。

In [17]:
Y_train = to_categorical(data.train_labels, 10)
Y_test = to_categorical(data.test_labels, 10)

print(type(Y_train))

<class 'numpy.ndarray'>


つまり用途としては

- `X_train`, `Y_train` => 学習させて、ウエイトの値を最適化法で求めて学習させる。

- `X_test`, `Y_test` => 学習させたデータを評価

となっています。結果として何を計算させたのかについては後で述べます。

## 3. Layerの構造

ここではネットワークを構築します。Tensorflowを触れた方はご存知かと思いますが、[ネットワークの構築と学習の際のデータのインプットは別個のものとして扱われます](https://qiita.com/hiroyuki827/items/509c2ac7735c8c16ad45)。Kerasも同様で、まずグラフを構築し、その後学習させる際にデータを流すという手順を踏みます。

まず、出力層(output layer)のニューロンは10個ですね。これは10個に分類したいことに対応しています。この数は特別な値なので、別に宣言しておきます。

In [18]:
num_classes = 10

今回は基本的な構造しか扱わないので、[`Sequential`モデル](https://keras.io/ja/models/sequential/)を扱います。インスタンス`model`を生成します。

In [19]:
model = Sequential()

これ以降はこのインスタンス`model`にlayerを追加していけばいいので、以下のように設定します。このネットワークの設定の仕方は、全く任意です。[Kerasのサンプルコード](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)を見ると、少し違ったコードが書かれています。実際、サンプルコードのほうを実行すると、より高い精度が得られます。今回は説明の都合上、低い精度を与えるニューラルネットワーク構造を考えてみます。

In [20]:
# CNN layer 1
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CNN layer 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

最後の`Activation`は必ず`softmax`です。一応参考書籍ではsoftmax層はいらないとか書いてあるのですが、まぁわかりやすさを考慮して付け加えておくことにします。(3行程度変わらない!)

以上のように構成したCNNは以下の構造をとっています。`Flatten()`によって`output shape`が１次元になっていることがわかりますね。

In [21]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

## 4. 学習させる
次に、上のように構築したニューラルネットワークに対して具体的にデータを入れて学習させます。

どのように学習させるかという情報は、[`compile`メソッド](https://keras.io/ja/models/sequential/#compile)で指定できます。`model`インスタンスに指定します。

In [22]:
model.compile(
loss='categorical_crossentropy', # 損失関数の設定
optimizer=Adam(lr=0.001), # 最適化法の指定
metrics=['accuracy'])

まずネットワークの最適化法を決めます。分類問題では`categorical_crossentropy`（交差エントロピー）を使います（上記書籍参照）。最適化法は`Adam`を使うことにしましょう。学習率は`lr=0.001`としておきます。（これはHyperparameterなので、結果を見ながら自分で決める必要があります。)

最適化法については: [Optimizer : 深層学習における勾配法について](http://qiita.com/tokkuman/items/1944c00415d129ca0ee9)

いよいよ学習させることにします。学習は`fit`メソッドで実行します。入力値と最終的な値の学習データを引数に入れます。バッチ数は大きい方が精度が高くなるのですが、メモリーが少ないとリソースエラーになってしまうので、程々の値としてここでは`128`にしておきます。(GPU上の計算でResource Errorになった場合は、この値が大きいことが原因であることが多いです。）エポック数も`40`です。バッチ数が128, エポック数40ということで、1エポックあたり全インプットデータを128ずつ取り出して計算を繰り返します。全データを取り終えたら1エポックの終了です。

また`validation_split`というのは、学習データの何パーセントを最終的なパラメータの設定に使うかを指定するものです。このデータにより、過学習しているかどうかを知ることができます。この設定のほかに個別に`validation_data`を設定すればより高精度の結果を得ることができるようですが、このチュートリアルではそれを目的としていないので、とりあえず10%を使うということにしておきます。(`validation_data`を使うときは`X_train`を適当に(`X_valid`, `Y_valid`などと分割する必要があります。)　

混乱を防ぐためにまとめておくと、

```
- X_train, Y_train => 学習データ. ウエイトとバイアスが決まったら学習終了
- X_test, Y_test => テストデータ. 学習後に分類できてるか確かめる。
- X_valid, Y_valid (もし`validation_data`使うなら) => 過学習のチェック. 適宜ハイパーパラメータを調整する。
```
となっています。以下で見るように、過学習をチェックするためには`val_loss`もしくは`val_acc`を追いかける必要があります。これらは上のいずれかを有効にすることで可能になります。

なお、メモリーの容量の関係から、`X_train`のデータをまるまる用いて計算させることはできません。その中のいくつかを`batch_size`で取り出して、計算させます。すべてのデータを一通り計算させたとき、それを1 epochと数えます。たとえば`batch_size=32`とすると、1 epoch内で`50000 (images) / 32 (images) = 1562`回だけ計算させることになります。1562回終わったら、1 epochです。

更に人間が決めるパラメータとして、何epoch学習させるかというものがあります。ここでは適当に`epochs=40`と設定しておきます。この回数は多すぎても時間がかかりすぎるだけですし（過学習が起きます）、少なすぎては学習が足りなくなってしまいます。さらにモデルによっても異なるので、どうすればいいかは少しづつ変えてみるしか無いのですが、Kerasでは`callbacks`の`EarlyStopping`が使えます。`EarlyStopping`の使い方は練習問題に回すとして、余力があれば色々変えて学習がどう進んでいくか見てみてください。

以上の情報を[`fit`メソッド](https://keras.io/ja/models/sequential/#fit)に入れます。

In [23]:
fit = model.fit(X_train, Y_train,
              batch_size=128,
              epochs=40,
              verbose=1,
              validation_split=0.1 # 今回は訓練データセットの10%をvalidationデータセットとして使う
                )

Train on 45000 samples, validate on 5000 samples
Epoch 1/40
45000/45000 [==============================] - 227s 5ms/step - loss: 1.6825 - acc: 0.3799 - val_loss: 1.3348 - val_acc: 0.5204
Epoch 2/40
45000/45000 [==============================] - 213s 5ms/step - loss: 1.2799 - acc: 0.5411 - val_loss: 1.0953 - val_acc: 0.6106
Epoch 3/40
45000/45000 [==============================] - 212s 5ms/step - loss: 1.0972 - acc: 0.6122 - val_loss: 0.9410 - val_acc: 0.6662
Epoch 4/40
45000/45000 [==============================] - 247s 5ms/step - loss: 0.9805 - acc: 0.6526 - val_loss: 0.8497 - val_acc: 0.7110
Epoch 5/40
45000/45000 [==============================] - 223s 5ms/step - loss: 0.8965 - acc: 0.6846 - val_loss: 0.8329 - val_acc: 0.7160
Epoch 6/40
45000/45000 [==============================] - 208s 5ms/step - loss: 0.8231 - acc: 0.7086 - val_loss: 0.7515 - val_acc: 0.7372
Epoch 7/40
45000/45000 [==============================] - 223s 5ms/step - loss: 0.7768 - acc: 0.7261 - val_loss: 0.6848 - v

最終的に学習させた結果として、およそ81.14%の精度で分類できることがわかりました。え？88.94%じゃないかって？その疑問はあとに取っておいてください。

<u>注意</u> val_accについては、訓練用のデータを用いて精度を見ているだけなので、テストデータ(そのモデルが実用可能かどうかを用いて評価したもの(model.evaluate)とは意味合いが異なります。詳しくは後述。

また、学習されたモデルがどの程度の精度を持ってテストデータを判定できるか(分類できるか)を評価することにします。これは[`evaluate`メソッド](https://keras.io/ja/models/sequential/#compile)で実現できます。

In [24]:
score = model.evaluate(X_test, Y_test,
                    verbose=0
                    )

print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.6653376616477966
Test accuracy: 0.7973


このように、学習させた結果を用いてテストデータを分類させると、75.35%の精度で正しく分類されることがわかりました。

なお、この結果を後から使いたいときは、以下のようにモデルを保存することができます。モデルの再利用については、付録Aを参照ください。

In [25]:
folder = 'results'
if not os.path.exists(folder):
    os.makedirs(folder)

model.save(os.path.join(folder, 'my_model.h5'))

## 5. 出力結果を見る

ここでは以上の結果をグラフを用いて見ていくことにします。dltパッケージがようやく本領発揮します。

### 5.1 分類チェック

dltでは訓練されたデータと、その予測値を各画像ごとに表示するコードが書かれています。

学習させたモデルから分類ラベルを取り出すには、[`predict`メソッド](https://keras.io/ja/models/sequential/#predict)が使えます。引数にはテストデータを指定します。さらにここでは最終的な値ではなく、`softmax`の出力するリストの最大値のラベルがほしいので、`np.argmax`でその添字を取り出します。各ラベルごとの精度を`preds`, 予測されるであろうラベルのarrayを`cls`とします。

**コメント** 私自身`predict`が実際に何をやっているのかわからなかったので、ここで詳しくまとめておきます。
最初に、`X_train, Y_train`を用いてネットワークを学習させました。この学習で80%の精度を得たわけですが、結果として得られたものは各層の重み、バイアスです。これが入力データ`X_train, Y_train`にあうように、そして**汎化性**(あとで述べます)を持つように決まったわけです。これが学習で得たものの実態です。次に、学習で得たネットワーク全体の構造を（文字通り）通して得られた画像(とか実体としてのデータ)がどう得られるかを見たい。(⇐今ここ) それをするのが`model.predict(X_test)`なんですね。`model`は学習されたネットワークを指定し、引数の`X_test`はテストデータをネットワークに通すために指定しているわけです。それで得られたものは`Yp`です。したがって、`Yp`は`Y_train`や`Y_test`と同じデータ構造(タプルとして)を持っていることになります。

まとめると、以下で比べるのは「学習されたネットワークを通した入力値(テストデータ)」と「実際のテストデータ」の違いです。もっというと、10個の要素を持つリストの中で、一番大きい確率を持つ要素の添字が一緒かどうかを見ています。(このあたりの内容は可視化しなくてもわかります: [自前の保存したモデルを用いて画像を分類してみる。](http://qiita.com/hiroyuki827/items/e6914b9b85f7bfe55078))

ともかく以下のように実装すれば

In [26]:
# predicted probabilities for the test set
preds = model.predict(X_test)
cls = model.predict_classes(X_test)

**注意**: ここでは新しいバージョンのKerasに実装された新しいメソッド`predict_classes`を用いることにします。ただしやっていることは`np.argmax(preds,axis=1)`と全く同じです。

ここから以下のデータが取り出せます。

In [27]:
# plot some test images along with the prediction
for i in range(10):
    dlt.utils.plot_prediction(
        preds[i],
        data.test_images[i],
        data.test_labels[i],
        data.classes,
        fname=os.path.join(folder, 'test-%i.png' % i))

<img src='results/test-0.png'/>
<img src='results/test-1.png'/>
<img src='results/test-2.png'/>
<img src='results/test-3.png'/>
<img src='results/test-4.png'/>

オレンジ色のついたラベルは、写真の正しいカテゴリーを示しており、バーの値は学習によるコンピュータの分類の精度を表しています。(どのくらいの確率で、その写真がそのカテゴリーに分類されるか。) テスト用の画像に対して、どの程度予測できているかがわかりますね。上２つの画像の分類は正しいですが、蛙(frog)に関しては分類が間違っていますね。

たとえば一つ目の画像ですが、正しいラベルは猫(cat)です。しかしながらコンピュータは数%の確率で犬(dog)だと考えています。まぁ90%以上の精度で猫だと言っているので間違いではありません。これは正しい分類!(人間でも間違えることありますよね?)

これとは対照的なのが二つ目の画像で、90%の確率で船(ship)だとあてていますが、10%程度でautomobileと分類してしまっています。。。

こんな感じで、テスト用の画像に対して、どの程度予測できているかがわかりますね。

**これが10個続きます**

Confusion Matrixのほうは以下のようになっています。

In [28]:
dlt.utils.plot_confusion_matrix(data.test_labels, cls, data.classes,
                                  title='confusion matrix',
                                  fname=os.path.join(folder, 'confusion_matrix.png'))

<img src='results/confusion_matrix.png'/>

と出力されます。これにより、学習によってどのカテゴリーに、より精度が高く(もしくは低く)分類されたかを知ることができます。上の結果をみると、犬と猫が精度が低いようです。。。

### 5.2 損失関数, 精度, 過学習

次に学習結果の確率と損失関数を見ます。以下のコードは実行する際は、いずれかをコメントアウトしてくださいね。

よく精度~%が得られたという意味で、精度が非常に重要視されるように見受けられるのですが、個人的にはloss functionの動きのほうがよっぽど大事だと思います。損失関数を見ずに確率だけを見ていた場合、その学習結果は**過学習**に陥っている可能性があるからです。どんな場合でも、損失関数の動きを見るようにしてください。

以下のコードでは、`fit.history['loss']`と`fit.history['val_loss']`をグラフにしています。`val_loss`, `val_acc`は`validation_split`を入れると自動的に生成されるラベルで、学習のインスタンス`fit`を用いて`fit.history[]`で呼び出せます。それぞれの出力結果は以下のようになっています。(精度についても`fit.history['acc']`と`fit.history['val_acc']`でできます。)

In [29]:
dlt.utils.plot_loss_and_accuracy(fit,  #model.fitのインスタンス
                                   fname=os.path.join(folder, 'cifar10-tutorial.png') #保存するファイル名とパス
                                  )

<img src='results/cifar10-tutorial.png'/>

#### Model Loss

まず損失関数の方を見ていきたいのですが、途中から`val_loss`が増加に転じていますね。これが過学習の兆候です！これ以上学習させてはいけません。
過学習は訓練用のデータに以上にフィットしていしまい、汎用化できなくなる現象です。

<img src='images/overfitting.png'/>

`X_train`のうち90%のデータを訓練させた結果、訓練用のデータの10%を使っても、76%の精度にとどまってしまうことを示しています。というわけで実際の精度は90%ではなく、76%(もしくはテストデータで測った72%)ということになります。過学習を止めるには、層を厚くするとか、Dropoutを入れるとか、いろいろな方法があります。ここではそんなに高い精度は求めていないので、議論しません。

Kerasでは`Early stopping`というメソッドが使えるので、これを使えば`val_loss`が増加に転じるところ、つまり最小値で学習を止めることができます。今回はデータを見ればそんなに学習させなくてもどこで止めるべきかわかるので使いませんでした。

損失関数を見れば過学習がわかると言いましたが、もちろん精度を見てもわかります。

#### Model Accuracy
右側のグラフを御覧ください。`val_acc`が途中から上昇を止めていますね。これも過学習が起きている証拠です。また`Early stopping`を用いる場合は、最大値になるところで止めれば良いことがわかりますね。

*精度だけ見ると早とちりしそうだけど、損失関数もみてやればじっくり見れる気がする...*

*参考*: このようなグラフからの判断は一概に方法があるわけではないので、非常にどこがおかしいのかを知るのは難しいのですが、個人的に以下のような記事を書いてみました: [ハイパーパラメータをいじってグラフを見てみる](http://qiita.com/hiroyuki827/items/213146d551a6e2227810)　

## 6. 全体のコード

これまで議論してきた全体のコードは以下のようになっています。(講義用からの転載なのでコメントは英語)

```
import numpy as np
import matplotlib.pyplot as plt
import dlt
import os

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import rmsprop
from keras.utils.np_utils import to_categorical

# ---------------------------------------------------------
# Load and preprocess data
# ---------------------------------------------------------
data = dlt.cifar.load_cifar10()

# plot some example images
dlt.utils.plot_examples(data, fname='examples.png')
print(data.train_images.shape)
print(data.train_labels.shape)
print(data.test_images.shape)
print(data.test_labels.shape)

# preprocess the data in a suitable way
# reshape the image matrices to vectors
#RGB 255 = white, 0 = black
X_train = data.train_images.reshape([-1, 32, 32, 3])
X_test = data.test_images.reshape([-1, 32, 32, 3])
print('%i training samples' % X_train.shape[0])
print('%i test samples' % X_test.shape[0])
print(X_train.shape)

# convert integer RGB values (0-255) to float values (0-1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# convert class labels to one-hot encodings
Y_train = to_categorical(data.train_labels, 10)
Y_test = to_categorical(data.test_labels, 10)

# Plot data distribution
dlt.utils.plot_distribution_data(Y=data.train_labels,
                                 dataset_name='y_train',
                                 classes=data.classes,
                                 fname='dist_train.png')
                                 
# ----------------------------------------------------------
# Model and training
# ----------------------------------------------------------


num_classes = 10

# model
model = Sequential()

# CNN layer 1
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CNN layer 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

model.compile(
loss='categorical_crossentropy',
optimizer=rmsprop(lr=0.0001, decay=1e-6),
metrics=['accuracy'])

fit = model.fit(X_train, Y_train,
              batch_size=32,
              epochs=50, #shouldn't be raised to 100, because the overfitting occurs.
              verbose=2,
              validation_split=0.1
                )

score = model.evaluate(X_test, Y_test,
                    verbose=0
                    )
print('Test score:', score[0])
print('Test accuracy:', score[1])

# ----------------------------------------------
# Some plots
# ----------------------------------------------

# make output directory
folder = 'results'
if not os.path.exists(folder):
    os.makedirs(folder)

model.save(os.path.join(folder, 'my_model.h5'))

# predicted probabilities for the test set
preds = model.predict(X_test)
cls = model.predict_classes(X_test)
                                  
# plot some test images along with the prediction
for i in range(10):
    dlt.utils.plot_prediction(
        preds[i],
        data.test_images[i],
        data.test_labels[i],
        data.classes,
        fname=os.path.join(folder, 'test-%i.png' % i))

# plot the confusion matrix
dlt.utils.plot_confusion_matrix(data.test_labels, cls, data.classes,
                                  title='confusion matrix',
                                  fname=os.path.join(folder, 'confusion matrix.png'))

# plot the loss and accuracy graph
dlt.utils.plot_loss_and_accuracy(fit,  #model.fitのインスタンス
                                   fname=os.path.join(folder, 'cifar10-tutorial.png') #保存するファイル名とパス
                                  )
```

## 7. 宿題

これまですべて完成したコードを示してきました。なので以下の問題は意味が無いのかもしれませんが、一通りやってみてください。

- どのようにすれば精度が高いCNNが得られるでしょうか？90%以上の精度(もちろん過学習なし)の実装をしてください。その場合、`Earlystopping`を用いて過学習が起きていないことをグラフとともに示し、議論してください。必要ならばその他のCallbackを使いなさい。(4 points)

- confusion matrixを出力し、それからわかることを議論してください。(3 points)

- 予測されたカテゴリーに対して、間違って分類された画像はいくつあるでしょうか？それらに共通してわかることはなにでしょうか？なぜそのような(分類ミス）が起こったのでしょうか？ (2 points)

- [Bonus] 付録A.3 を参考にして、カテゴリー外の画像ならどうなるか議論しなさい。また、ImageNetを用いた画像を判定するコードを公式ドキュメントを参考にして実装し、その画像が正しく分類されることを確認しなさい。 (10 points)

## 8. 終わりに

少々中途半端なチュートリアルになってしまいましたが、CNNやKerasの魅力をわかっていただけるとこれ以上の喜びはありません。もっともっとKerasユーザーが増え、Kerasの発展に貢献していくことを願います。

なお、実際は１からネットワークを構築して学習せずとも、すでに学習させたモデルのネットワークを用いて学習させれば、より少ないリソースで高い精度の結果を得ることができます。これを**転移学習**と言います。またの機会にこの学習について触れることにします。